## Installation 

If you have not installed snowflake-snowpark-python you can install it here. Make sure you are in the right conda environment.

If you are not sure you can try:

```bash
! which python
```

In [1]:
! which python

/home/zghaffar/miniconda3/envs/notebook_env/bin/python


Install the library

In [1]:
!conda install snowflake-snowpark-python -y

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/zghaffar/miniconda3/envs/dev1

  added / updated specs:
    - snowflake-snowpark-python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asn1crypto-1.5.1           |  py311h06a4308_0         220 KB
    certifi-2024.7.4           |  py311h06a4308_0         159 KB
    cloudpickle-2.2.1          |  py311h06a4308_0          51 KB
    cryptography-42.0.5        |  py311hdda0065_1         2.1 MB
    filelock-3.13.1            |  py311h06a4308_0          24 KB
    pyjwt-2.8.0                |  py311h06a4308_0          85 KB
    pyopenssl-24.0.0           |  py311h06a4308_0         124 KB
    snowflake-connector-python-3.11.0|  py311h6a678d5_0         1.3 MB
    snowflake-snowpark-python-1.18.0|  py311h06a4308_0         2.1 M

# THE **E**
## Snowpark Usage - Example

In [42]:
from snowflake.snowpark import Session

Example of how the `snow.cfg` file looks like

```yaml
[SNOW]
ACCOUNT=WPA36811
USER=tarek
PASSWORD=yourpassowrd
WAREHOUSE=COMPUTE_WH
DATABASE=TECHCATALYST_DE
SCHEMA=PUBLIC
ROLE=DE
```

In [43]:
import configparser

config = configparser.ConfigParser()
config.read('snow.cfg')
config.sections()

['SNOW']

Create a dictionary (key-value) of the parameters

In [44]:
params1 = dict(config['SNOW'])

Passing the parameters as a dictionary to Session

In [45]:
session1 = Session.builder.configs(params1).create()

In [46]:
print(session1.get_current_user())
print(session1.get_current_database())
print(session1.get_current_schema())

"Zayd_Ghaffar"
"TECHCATALYST_DE"
"ZAYD"


Example using the `table` methods to call a specific table. In this example I am calling the `INS_ACCIDENTS` table

In [47]:
accidents = session1.table("INS_ACCIDENTS")
accident_type = session1.table("INS_ACCIDENT_TYPE")
body_style = session1.table("INS_BODY_STYLE")
gmstatus = session1.table("INS_GENDER_MARITAL_STATUS")
coverage = session1.table("INS_INSURANCE_COVERAGE")
policyholder = session1.table("INS_POLICYHOLDER")
states = session1.table("INS_STATES")
v = session1.table("INS_VEHICLE")
vu = session1.table("INS_VEHICLE_USE")
vuc = session1.table("INS_VEHICLE_USE_CODE")



accidents.show(5)

-------------------------------------------------------------------------------------------------------------------------------------------------------
|"ACCIDENT_ID"  |"POLICYHOLDER_ID"  |"VEHICLE_ID"  |"ACCIDENT_TYPE"  |"ACCIDENT_DATE"  |"ESTIMATED_COST"  |"ACTUAL_REPAIR_COST"  |"AT_FAULT"  |"DUI"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |4333               |176           |3                |2022-03-30       |6694              |7480                  |True        |True   |
|2              |4547               |6391          |4                |2023-09-27       |2190              |3355                  |True        |False  |
|3              |6686               |6974          |7                |2022-06-13       |4995              |7123                  |False       |False  |
|4              |1300               |4037          |7                |2020-01-21       |

Note, `accidents` is a Snowpark object (table). There is a difference between Snowpark DataFrames and Panda DataFrames. For now, you will convert to a Pandas DataFrame to do your transformations. Once done, you will convert back to a Snowpark DataFrame and write to the Database.

In [48]:
accidents = session1.table("INS_ACCIDENTS")
accident_type = session1.table("INS_ACCIDENT_TYPE")
body_style = session1.table("INS_BODY_STYLE")
gmstatus = session1.table("INS_GENDER_MARITAL_STATUS")
coverage = session1.table("INS_INSURANCE_COVERAGE")
policyholder = session1.table("INS_POLICYHOLDER")
states = session1.table("INS_STATES")
v = session1.table("INS_VEHICLES")
vu = session1.table("INS_VEHICLE_USE")
vuc= session1.table("INS_VEHICLE_USE_CODE")



accidents.show(5)

-------------------------------------------------------------------------------------------------------------------------------------------------------
|"ACCIDENT_ID"  |"POLICYHOLDER_ID"  |"VEHICLE_ID"  |"ACCIDENT_TYPE"  |"ACCIDENT_DATE"  |"ESTIMATED_COST"  |"ACTUAL_REPAIR_COST"  |"AT_FAULT"  |"DUI"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |4333               |176           |3                |2022-03-30       |6694              |7480                  |True        |True   |
|2              |4547               |6391          |4                |2023-09-27       |2190              |3355                  |True        |False  |
|3              |6686               |6974          |7                |2022-06-13       |4995              |7123                  |False       |False  |
|4              |1300               |4037          |7                |2020-01-21       |

In [49]:
type(accidents)

snowflake.snowpark.table.Table

In [50]:
accidents_df = accidents.to_pandas()
accident_type_df = accident_type.to_pandas()
body_style_df= body_style.to_pandas()
gmstatus_df = gmstatus.to_pandas()
coverage_df = coverage.to_pandas()
ph_df = policyholder.to_pandas()
states_df = states.to_pandas()
v_df = v.to_pandas()
vu_df = vu.to_pandas()
vuc_df = vuc.to_pandas()

In [51]:
# accidents_df.info()
# vuc_df.info()
v_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   VEHICLE_ID       10000 non-null  int16
 1   POLICYHOLDER_ID  10000 non-null  int16
 2   BODY_STYLE_CODE  10000 non-null  int8 
 3   YEAR             10000 non-null  int16
dtypes: int16(3), int8(1)
memory usage: 68.5 KB


# The **T**
## Your Work Here (Transformations)

In [52]:
import pandas as pd
dim_at_df = pd.DataFrame()
dim_at_df['ACCIDENT_TYPE_ID'] = accident_type_df['ACCIDENT_TYPE_CODE']
dim_at_df['ACCIDENT_TYPE'] = accident_type_df['ACCIDENT_TYPE']

In [53]:
dim_bs_df = pd.DataFrame()
dim_bs_df['BODY_STYLE'] = body_style_df['BODY_STYLE']
dim_bs_df['BODY_STYLE_ID'] = body_style_df['BODY_STYLE_CODE']

In [54]:
dim_ph_df = pd.DataFrame()
dim_ph_df['ADDRESS'] = ph_df['ADDRESS']
dim_ph_df['FIRST_NAME'] = ph_df['FIRST_NAME']
dim_ph_df['LAST_NAME'] = ph_df['LAST_NAME']
dim_ph_df['POLICYHOLDER_ID'] = ph_df['POLICYHOLDER_ID']

In [55]:
dim_st_df = pd.DataFrame()
dim_st_df['STATE'] = states_df['STATE']
dim_st_df['STATE_ID'] = states_df['STATE_CODE']

In [56]:
dim_gm_df = pd.DataFrame()
dim_gm_df['GENDER_MARITALSTATUS_ID'] = gmstatus_df['GENDER_MARITAL_STATUS_CODE']
dim_gm_df['GENDER_MARITAL_STATUS'] = gmstatus_df['GENDER_MARITAL_STATUS']

In [57]:
dim_vu_df = pd.DataFrame()
dim_vu_df['VEHICLE_USE'] = vu_df['USE_ID']
dim_vu_df['VEHICLE_USECODE_ID'] = vu_df['USE_CODE']

In [60]:
# accidents_df = accidents.to_pandas()
# accident_type_df = accident_type.to_pandas()
# body_style_df= body_style.to_pandas()
# gmstatus_df = gmstatus.to_pandas()
# coverage_df = coverage.to_pandas()
# ph_df = policyholder.to_pandas()
# states_df = states.to_pandas()
# v_df = v.to_pandas()
# vu_df = vu.to_pandas()
# vuc_df = vuc.to_pandas()
fa_df = pd.DataFrame()
 
fa_df = pd.merge(accidents_df, accident_type_df.drop(columns='ACCIDENT_TYPE'), left_on = 'ACCIDENT_TYPE', right_on='ACCIDENT_TYPE_CODE', how='inner')
fa_df = pd.merge(fa_df, ph_df, on = 'POLICYHOLDER_ID', how="inner")
fa_df = pd.merge(fa_df, v_df.drop(columns='POLICYHOLDER_ID'), on = 'VEHICLE_ID', how="inner")
fa_df = pd.merge(fa_df, states_df, on = 'STATE_CODE', how="inner")
fa_df = pd.merge(fa_df, coverage_df, on = 'POLICYHOLDER_ID', how="inner")
fa_df = pd.merge(fa_df, gmstatus_df.drop(columns='GENDER_MARITAL_STATUS'), left_on = 'GENDER_MARITAL_STATUS', right_on='GENDER_MARITAL_STATUS_CODE', how="inner")
fa_df = pd.merge(fa_df, body_style_df, on = 'BODY_STYLE_CODE', how="inner")
fa_df = pd.merge(fa_df, vu_df, on = 'VEHICLE_ID', how="inner")


# fa_df.head()
fa_order_df = fa_df[['ACCIDENT_DATE', 'ACCIDENT_ID', 'ACCIDENT_TYPE_CODE', 'ACTUAL_REPAIR_COST', 'AT_FAULT', 'BODY_STYLE_CODE', 'COVERAGE_STATUS', 'ESTIMATED_COST', 'GENDER_MARITAL_STATUS_CODE', 'DUI', 'BIRTHDATE', 'POLICYHOLDER_ID', 'STATE_CODE', 'VEHICLE_ID', 'USE_CODE','YEAR']]

fa_order_df.head()
fa_order_df.columns

Index(['ACCIDENT_DATE', 'ACCIDENT_ID', 'ACCIDENT_TYPE_CODE',
       'ACTUAL_REPAIR_COST', 'AT_FAULT', 'BODY_STYLE_CODE', 'COVERAGE_STATUS',
       'ESTIMATED_COST', 'GENDER_MARITAL_STATUS_CODE', 'DUI', 'BIRTHDATE',
       'POLICYHOLDER_ID', 'STATE_CODE', 'VEHICLE_ID', 'USE_CODE', 'YEAR'],
      dtype='object')

In [63]:
fa_order_df.rename(columns= {'ACCIDENT_TYPE_CODE':'ACCIDENT_TYPE_ID', 'BODY_STYLE_CODE':'BODY_STYLE_ID', 'GENDER_MARITAL_STATUS_CODE':'GENDER_MARITAL_STATUS_ID', 'DUI':'IS_DUI', 'BIRTHDATE':'PLACEHOLDER_BIRTHDATE', 'STATE_CODE':'STATE_ID', 'USE_CODE':'VEHICLE_USECODE_ID', 'YEAR':'VEHICLE_YEAR'})

fa_final = fa_order_df.drop_duplicates()
fa_final = fa_final.reset_index(drop = True)

fa_final.shape

(6383, 16)

# THE **L**
## Write to Snowflak

To avoid mistakes, make sure you use your schema. 

In [64]:
yourschema = 'ZAYD'
location = f'techcatalyst_de.{yourschema}'
print(location)

techcatalyst_de.ZAYD


Convert the Pandas DataFrame into a Snowpark DataFrame

In [66]:
fa_sdf = session1.create_dataframe(fa_final)
at_sdf = session1.create_dataframe(dim_at_df)
bs_sdf = session1.create_dataframe(dim_bs_df)
ph_sdf = session1.create_dataframe(dim_ph_df)
st_sdf = session1.create_dataframe(dim_st_df)
gm_sdf = session1.create_dataframe(dim_gm_df)
vu_sdf = session1.create_dataframe(dim_vu_df)



In [133]:
print(type(accidents_df))
print(type(accidents_sdf))

<class 'pandas.core.frame.DataFrame'>
<class 'snowflake.snowpark.table.Table'>


When using the `write.mode()` method there are different options:

* ”`append`”: Append data of this DataFrame to the existing table. Creates a table if it does not exist.

* ”`overwrite`”: Overwrite the existing table by dropping old table.

* ”`truncate`”: Overwrite the existing table by truncating old table.

* ”`errorifexists`”: Throw an exception if the table already exists.

* ”`ignore`”: Ignore this operation if the table already exists.

* Default value is “`errorifexists`”.

In [68]:
# example using overwrite

# accidents_sdf.write.mode("overwrite").save_as_table(f"{location}.tatwan_test")
fa_sdf = session1.create_dataframe(fa_final)
fa_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_FACT_ACCIDENT")
at_sdf = session1.create_dataframe(dim_at_df)
at_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_DIM_ACCIDENT_TYPE")
bs_sdf = session1.create_dataframe(dim_bs_df)
bs_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_BODY_STYLE")
gm_sdf = session1.create_dataframe(dim_gm_df)
gm_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_DIM_GENDER_MARITAL")
ph_sdf = session1.create_dataframe(dim_ph_df)
ph_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_DIM_POLICYHOLDER")
st_sdf = session1.create_dataframe(dim_st_df)
st_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_DIM_STATES")
vu_sdf = session1.create_dataframe(dim_vu_df)
vu_sdf.write.mode("overwrite").save_as_table(f"{location}.PY_VEHICLE_USE")